In [3]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
rough_data=pd.read_csv('dataset.csv')
rough_data.head(10)

,total questions,correct questions,Performance,Job Profile Name,correctness
0,5,1,Very Poor,Full Stack Developer,20.00
1,1,5,can't predict,Full Stack Developer,500.00
2,9,3,Poor,Full Stack Developer,33.33
3,5,6,can't predict,Full Stack Developer,120.00
4,8,5,Above Average,Database Administrator,62.50
5,8,10,can't predict,Full Stack Developer,125.00
6,3,9,can't predict,Artificial Intelligence (AI) Engineer,300.00
7,8,10,can't predict,Cybersecurity Analyst,125.00
8,1,0,Needs Improvement,Data Analyst,0.00
9,3,7,can't predict,Database Administrator,233.33


In [4]:
print(rough_data.describe())

       total questions  correct questions  correctness
count      2000.000000        2000.000000  2000.000000
mean          5.588000           4.941000   140.457450
std           2.866944           3.158244   175.632592
min           1.000000           0.000000     0.000000
25%           3.000000           2.000000    40.000000
50%           6.000000           5.000000    90.000000
75%           8.000000           8.000000   160.000000
max          10.000000          10.000000  1000.000000


In [5]:
filter=(rough_data['correct questions']>rough_data['total questions'])
# print(rough_data[filter].head(5))
data=rough_data[~filter]
print(data.head(5))

    total questions  correct questions        Performance  \
0                 5                  1          Very Poor   
2                 9                  3               Poor   
4                 8                  5      Above Average   
8                 1                  0  Needs Improvement   
10                4                  4          Excellent   

          Job Profile Name  correctness  
0     Full Stack Developer        20.00  
2     Full Stack Developer        33.33  
4   Database Administrator        62.50  
8             Data Analyst         0.00  
10            Data Analyst       100.00  


In [6]:
#code to do label encoding but manual function to understand the performance from 0 to 9 i.e Nedds improvement to excellent
def map_scores_to_labels(score):
    if 0 <= score <= 9:
        return 0
    elif 10 <= score <= 19:
        return 1
    elif 20 <= score <= 29:
        return 2
    elif 30 <= score <= 39:
        return 3
    elif 40 <= score <= 49:
        return 4
    elif 50 <= score <= 59:
        return 5
    elif 60 <= score <= 69:
        return 6
    elif 70 <= score <= 79:
        return 7
    elif 80 <= score <= 89:
        return 8
    elif 90 <= score <= 100:
        return 9
    else:
        return -1  # value to signify 'Out of Range' or can't predict
data['Target_performance'] = data['correctness'].apply(map_scores_to_labels)
# Display the modified DataFrame
# print(data['Target_performance'].isna)

C:\Users\dalvi\AppData\Local\Temp\ipykernel_20888\854457378.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Target_performance'] = data['correctness'].apply(map_scores_to_labels)


In [9]:
features = ['total questions', 'correct questions', 'correctness', 'Job Profile Name']
target = 'Target_performance'
# Encoding categorical variables
label_encoders = {}
for col in ['Job Profile Name']:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

C:\Users\dalvi\AppData\Local\Temp\ipykernel_20888\3222501537.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = label_encoders[col].fit_transform(data[col])


In [10]:
# Trainng starts here
X = data[features]
y = data[target]
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y.isnull
# Initialize and train the model (Random Forest Classifier as an example)
model =RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [11]:
from sklearn.metrics import classification_report
# Make predictions on the test set
predictions = model.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
class_report = classification_report(y_test, predictions)
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{class_report}')


Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        44
           1       1.00      1.00      1.00        19
           2       1.00      1.00      1.00        25
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        14
           5       1.00      1.00      1.00        22
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00        19
           8       1.00      1.00      1.00        25
           9       1.00      1.00      1.00        48

    accuracy                           1.00       242
   macro avg       1.00      1.00      1.00       242
weighted avg       1.00      1.00      1.00       242



In [20]:
# Prediction using new data
new_data = pd.DataFrame({
    'total questions': [50],
    'correct questions': [10],
    'correctness': [20],
    'Job Profile Name': ['Full Stack Developer']
})
# Transform categorical columns in new data
for col in [ 'Job Profile Name']:
    new_data[col] = label_encoders[col].transform(new_data[col])

# Predict 'Performance' for new data
predicted_performance = model.predict(new_data)
if(predicted_performance==0):
    print(f'Predicted Performance for new data: Needs Improvement')
elif(predicted_performance==1):
    print(f'Predicted Performance for new data: Extremely Poor')
elif(predicted_performance==2):
    print(f'Predicted Performance for new data: Very Poor')
elif(predicted_performance==3):
    print(f'Predicted Performance for new data:Poor')
elif(predicted_performance==4):
    print(f'Predicted Performance for new data: Below Average')
elif(predicted_performance==5):
    print(f'Predicted Performance for new data: Average')
elif(predicted_performance==6):
    print(f'Predicted Performance for new data:Above Average')
elif(predicted_performance==7):
    print(f'Predicted Performance for new data: Good')
elif(predicted_performance==8):
    print(f'Predicted Performance for new data:Very Good')
elif(predicted_performance==9):
    print(f'Predicted Performance for new data:Excellent')


Predicted Performance for new data: Very Poor
